## Looking at CAM Budgets

First we set up out environment as before:


In [98]:
# Add ldcpy root to system path
import sys

sys.path.insert(0, '../../../')

# Import ldcpy package
# Autoreloads package everytime the package is called, so changes to code will be reflected in the notebook if the above sys.path.insert(...) line is uncommented.
%load_ext autoreload
%autoreload 2
import ldcpy

# display the plots in this notebook
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Collections for variables needed for budgets


In [99]:
fsnt_col = ldcpy.open_datasets(
    ["FSNT"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/FSNT.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/FSNT.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)
flnt_col = ldcpy.open_datasets(
    ["FLNT"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/FLNT.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/FLNT.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)
precc_col = ldcpy.open_datasets(
    ["PRECC"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/PRECC.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/PRECC.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)
precl_col = ldcpy.open_datasets(
    ["PRECL"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/PRECL.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/PRECL.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)
fsns_col = ldcpy.open_datasets(
    ["FSNS"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/FSNS.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/FSNS.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)
flns_col = ldcpy.open_datasets(
    ["FLNS"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/FLNS.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/FLNS.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)
shflx_col = ldcpy.open_datasets(
    ["SHFLX"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/SHFLX.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/SHFLX.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)
lhflx_col = ldcpy.open_datasets(
    ["LHFLX"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/LHFLX.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/LHFLX.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)
qflx_col = ldcpy.open_datasets(
    ["QFLX"],
    [
        "/Users/abaker/alli/code-repo/test-data/budgets/orig/QFLX.monthly.10yrs.nc",
        "/Users/abaker/alli/code-repo/test-data/budgets/lossy/QFLX.monthly.10yrs.nc",
    ],
    ["orig", "lossy"],
)

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05



In [122]:
ldcpy.cam_restom(fsnt_col, flnt_col, [["orig", "lossy"], ["orig", "lossy"]])

values =  [-27.42708969 -27.42694283]
percent rel. difference =  [0.00053548]


In [123]:
ldcpy.cam_precip(precc_col, precl_col, [["orig", "lossy"], ["orig", "lossy"]])

values =  [2.83991248e-08 2.83988282e-08]
percent rel. difference =  [0.00104458]


In [124]:
ldcpy.cam_ep(
    qflx_col, precc_col, precl_col, [["orig", "lossy"], ["orig", "lossy"], ["orig", "lossy"]]
)

values =  [4.70243435 4.70238279]
percent rel. difference =  [0.00109638]


In [125]:
ldcpy.cam_ressurf(
    fsns_col,
    flns_col,
    shflx_col,
    lhflx_col,
    [["orig", "lossy"], ["orig", "lossy"], ["orig", "lossy"], ["orig", "lossy"]],
)

values =  [-1.15766227 -0.97469544]
percent rel. difference =  [15.80485439]


In [119]:
ldcpy.cam_ressurf(
    fsns_col,
    flns_col,
    shflx_col,
    lhflx_col,
    [["orig", "lossy"], ["orig", "lossy"], ["orig", "lossy"], ["orig", "lossy"]],
)

values =  [-1.15766227 -0.97469544]
percent difference =  [15.80485439]


In [121]:
(-1.157 + 0.97)

-0.18700000000000006

In [137]:
lhflx0 = lhflx_col.isel(time=0)
ldcpy.compare_stats(lhflx_col, "LHFLX", ["orig", "lossy"])

Warning - this data set has a time dimension - examoning slice 0 only...


,orig,lossy
mean,64.85,64.671
variance,4447.5,4423.2
standard deviation,66.69,66.508
min value,-5.7709,-5.75
max value,360.22,360
probability positive,0.82897,0.82897
number of zeros,0,0


,lossy
max abs diff,1.9836
min abs diff,0
mean abs diff,0.17892
mean squared diff,0.031799
root mean squared diff,0.30147
normalized root mean squared diff,0.0008237
normalized max pointwise error,0.0054199
pearson correlation coefficient,1
ks p-value,0.83323
spatial relative error(% > 0.0001),98.139


In [138]:
ldcpy.compare_stats(lhflx0, "LHFLX", ["orig", "lossy"])

,orig,lossy
mean,64.85,64.671
variance,4447.5,4423.2
standard deviation,66.69,66.508
min value,-5.7709,-5.75
max value,360.22,360
probability positive,0.82897,0.82897
number of zeros,0,0


,lossy
max abs diff,1.9836
min abs diff,0
mean abs diff,0.17892
mean squared diff,0.031799
root mean squared diff,0.30147
normalized root mean squared diff,0.0008237
normalized max pointwise error,0.0054199
pearson correlation coefficient,1
ks p-value,0.83323
spatial relative error(% > 0.0001),98.139


In [135]:
b = lhflx_col['LHFLX']
if 'time' in b.dims:
    print('yes')

yes


In [134]:
lhflx0

<xarray.Dataset>
Dimensions:     (collection: 2, lat: 192, lon: 288)
Coordinates:
  * lat         (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon         (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    time        object 2006-02-01 00:00:00
  * collection  (collection) <U5 'orig' 'lossy'
Data variables:
    LHFLX       (collection, lat, lon) float32 dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:      CF-1.0
    source:           CAM
    case:             b.e11.BRCP85C5CNBDRD.f09_g16.031
    title:            UNSET
    logname:          mickelso
    host:             ys1023
    Version:          $Name$
    revision_Id:      $Id$
    initial_file:     b.e11.B20TRC5CNBDRD.f09_g16.031.cam.i.2006-01-01-00000.nc
    topography_file:  /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/USGS-gtop...
    history:          Thu Apr 22 11:46:10 2021: ncks -d time,0,119,1 b.e11.BR...
    NCO:              netCDF Operators version 4.9.5 (Homepage = http://nco.s...

In [104]:
fsnt_col.time.dt.year

<xarray.DataArray 'year' (time: 120)>
array([2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006,
       2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007,
       2007, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008,
       2008, 2008, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009,
       2009, 2009, 2009, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010,
       2010, 2010, 2010, 2010, 2011, 2011, 2011, 2011, 2011, 2011, 2011,
       2011, 2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012, 2012, 2012,
       2012, 2012, 2012, 2012, 2012, 2012, 2013, 2013, 2013, 2013, 2013,
       2013, 2013, 2013, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2014,
       2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2015, 2015, 2015,
       2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2016])
Coordinates:
  * time     (time) object 2006-02-01 00:00:00 ... 2016-01-01 00:00:00

In [105]:
spy = 12

In [22]:
int(23 / spy)

1

In [25]:
fsnt_data = []
fsnt = fsnt_col['FSNT']

fsnt_data.append(fsnt.sel(collection="orig"))

In [33]:
month_length = fsnt.time.dt.days_in_month
month_length

<xarray.DataArray 'days_in_month' (time: 120)>
array([28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30,
       31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30,
       31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30,
       31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30,
       31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28,
       31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31,
       31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31,
       31])
Coordinates:
  * time     (time) object 2006-02-01 00:00:00 ... 2016-01-01 00:00:00

In [55]:
month_length = fsnt_col.time.dt.days_in_month[0:12]
print(month_length)
wts = month_length.groupby('time.year') / month_length.groupby('time.year').sum()

<xarray.DataArray 'days_in_month' (time: 12)>
array([28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31])
Coordinates:
  * time     (time) object 2006-02-01 00:00:00 ... 2007-01-01 00:00:00


In [57]:
wts

<xarray.DataArray 'days_in_month' (time: 12)>
array([0.08383234, 0.09281437, 0.08982036, 0.09281437, 0.08982036,
       0.09281437, 0.09281437, 0.08982036, 0.09281437, 0.08982036,
       0.09281437, 1.        ])
Coordinates:
  * time     (time) object 2006-02-01 00:00:00 ... 2007-01-01 00:00:00
    year     (time) int64 2006 2006 2006 2006 2006 ... 2006 2006 2006 2006 2007